### 2.1 環境設定

In [ ]:
#%pip install openai
#%pip install python-dotenv

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.2 -> 25.0.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [1]:
from openai import OpenAI
import os
from dotenv import load_dotenv

# Load enviroment variable
path_env="C:\\Users\\Yuichi Katogi\learning-repo\\.env"
load_dotenv(path_env)

client = OpenAI(
  api_key=os.getenv("OPENAI_API_KEY")
)

### 2.2 実行してみる

In [ ]:
completion = client.chat.completions.create(
  model="gpt-4o-mini",
  messages=[
    {"role": "user", "content": "write a haiku about ai"}
  ]
)

print(completion.choices[0].message.content)


### 2.3 JSON形式で出力してみる

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"}
    ]
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-Azh3puquDF4RWU9W8pt7WrW7P7H1S",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことを話しましょうか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1739266897,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_72ed7ab54c",
  "usage": {
    "completion_tokens": 26,
    "prompt_tokens": 25,
    "total_tokens": 51,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


### 2.4 roleでassistantを使う

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"},
        {"role": "assistant", "content": "こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことを話しましょうか？"},
        {"role": "user", "content": "私の名前がわかりますか？"}
    ]
)

print(response.to_json(indent=2))

{
  "id": "chatcmpl-AzhAPlOOZjp3G1InRNXGredm0QpW8",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "はい、あなたの名前をジョンさんと教えていただきました。何か他にお話ししたいことはありますか？",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1739267305,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_72ed7ab54c",
  "usage": {
    "completion_tokens": 31,
    "prompt_tokens": 67,
    "total_tokens": 98,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}


### 2.5 ストリーミング形式で出力する

In [ ]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role": "system", "content": "You are a helpful assistant."},
        {"role": "user", "content": "こんにちは！私はジョンと言います！"}
    ],
    stream=True
)

for chunk in response:
    content = chunk.choices[0].delta.content
    if content is not None:
        print(content, end="", flush=True)

こんにちは、ジョンさん！お会いできて嬉しいです。今日はどんなことをお話ししましょうか？

### 2.6 JSON形式で登場人物を出力させる

In [2]:
response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "system",
            "content": '人物一覧を次のJSON形式で出力してください。\n{"people": ["aaa", "bbb"]}',
        },
        {
            "role": "user",
            "content": "昔々あるところにおじいさんとおばあさんがいました",
        },
    ],
    response_format={"type": "json_object"},
)

print(response.choices[0].message.content)

{"people": ["おじいさん", "おばあさん"]}


### 2.7 画像の説明を依頼する

In [3]:
# 説明して欲しい画像をインプットする
image_url = "https://raw.githubusercontent.com/yoshidashingo/langchain-book/main/assets/cover.jpg"

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {
            "role": "user",
            "content": [
                {"type": "text", "text": "画像を説明してください。"},
                {"type": "image_url", "image_url": {"url": image_url}},
            ],
        }
    ],
)

print(response.choices[0].message.content)

この画像は、書籍の表紙です。タイトルは「ChatGPT/LangChainによるチャットシステム構築[実践]入門」となっており、著者の名前が書かれています。表紙のデザインにはカラフルなオウムのイラストがあり、背景は鮮やかな色合いになっています。「ChatGPT」のロゴも目立つように配置されています。全体的に、テクノロジーやプログラミングに関連した内容を示唆しています。


### 2.8 Function calling

In [ ]:
# ここでAPIを読み込む。テキストではデータ作ってダミーデータで結果を出している

import json

def get_current_weather(location, unit="fahrenheit"):
    if "tokyo" in location.lower():
        return json.dumps({"location": "Tokyo", "temperature": "10", "unit": unit})
    elif "san francisco" in location.lower():
        return json.dumps(
            {"location": "San Francisco", "temperature": "72", "unit": unit}
        )
    elif "paris" in location.lower():
        return json.dumps({"location": "Paris", "temperature": "22", "unit": unit})
    else:
        return json.dumps({"location": location, "temperature": "unknown"})

In [5]:
tools = [
    {
        "type": "function",
        "function": {
            "name": "get_current_weather",
            "description": "Get the current weather in a given location",
            "parameters": {
                "type": "object",
                "properties": {
                    "location": {
                        "type": "string",
                        "description": "The city and state, e.g. San Francisco, CA",
                    },
                    "unit": {"type": "string", "enum": ["celsius", "fahrenheit"]},
                },
                "required": ["location"],
            },
        },
    }
]

In [6]:
messages = [
    {"role": "user", "content": "東京の天気はどうですか？"},
]

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
    tools=tools,
)
print(response.to_json(indent=2))

{
  "id": "chatcmpl-B1Syx09anxy1cccDAj2lLYmdxSNna",
  "choices": [
    {
      "finish_reason": "tool_calls",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": null,
        "refusal": null,
        "role": "assistant",
        "tool_calls": [
          {
            "id": "call_YpbdMwPV5chVeynV9Gx6BKGu",
            "function": {
              "arguments": "{\"location\":\"Tokyo, JP\"}",
              "name": "get_current_weather"
            },
            "type": "function"
          }
        ]
      }
    }
  ],
  "created": 1739689435,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_00428b782a",
  "usage": {
    "completion_tokens": 18,
    "prompt_tokens": 81,
    "total_tokens": 99,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_d

In [7]:
response_message = response.choices[0].message
messages.append(response_message.to_dict())

In [8]:
available_functions = {
    "get_current_weather": get_current_weather,
}

# 使いたい関数は複数あるかもしれないのでループ
for tool_call in response_message.tool_calls:
    # 関数を実行
    function_name = tool_call.function.name
    function_to_call = available_functions[function_name]
    function_args = json.loads(tool_call.function.arguments)
    function_response = function_to_call(
        location=function_args.get("location"),
        unit=function_args.get("unit"),
    )
    print(function_response)

    # 関数の実行結果を会話履歴としてmessagesに追加
    messages.append(
        {
            "tool_call_id": tool_call.id,
            "role": "tool",
            "name": function_name,
            "content": function_response,
        }
    )

{"location": "Tokyo", "temperature": "10", "unit": null}


In [9]:
print(json.dumps(messages, ensure_ascii=False, indent=2))

[
  {
    "role": "user",
    "content": "東京の天気はどうですか？"
  },
  {
    "content": null,
    "refusal": null,
    "role": "assistant",
    "tool_calls": [
      {
        "id": "call_YpbdMwPV5chVeynV9Gx6BKGu",
        "function": {
          "arguments": "{\"location\":\"Tokyo, JP\"}",
          "name": "get_current_weather"
        },
        "type": "function"
      }
    ]
  },
  {
    "tool_call_id": "call_YpbdMwPV5chVeynV9Gx6BKGu",
    "role": "tool",
    "name": "get_current_weather",
    "content": "{\"location\": \"Tokyo\", \"temperature\": \"10\", \"unit\": null}"
  }
]


In [10]:
second_response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=messages,
)
print(second_response.to_json(indent=2))

{
  "id": "chatcmpl-B1Szo8WGbtnRNxnzszL6JPhpNMPbQ",
  "choices": [
    {
      "finish_reason": "stop",
      "index": 0,
      "logprobs": null,
      "message": {
        "content": "東京の現在の天気は気温10度です。天候に関するもっと具体的な情報が必要な場合は教えてください。",
        "refusal": null,
        "role": "assistant"
      }
    }
  ],
  "created": 1739689488,
  "model": "gpt-4o-mini-2024-07-18",
  "object": "chat.completion",
  "service_tier": "default",
  "system_fingerprint": "fp_13eed4fce1",
  "usage": {
    "completion_tokens": 33,
    "prompt_tokens": 59,
    "total_tokens": 92,
    "completion_tokens_details": {
      "accepted_prediction_tokens": 0,
      "audio_tokens": 0,
      "reasoning_tokens": 0,
      "rejected_prediction_tokens": 0
    },
    "prompt_tokens_details": {
      "audio_tokens": 0,
      "cached_tokens": 0
    }
  }
}
